# ColdFusion
### GPR-LIDAR-FUSION
7/12/22

Query the SnowEx database to extract ground based snow depth measurements, snow pit density, GPR tracks, and Lidar point clounds where there is spatial and temporal overlap in a defined area.

Use the lidar snow depth measurements to inform the GPR density estimate to get SWE.

Test these density/SWE estimates against snowpit values

In [ ]:
# Imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas
from snowexsql.db import get_table_attributes

import datetime 
from sqlalchemy import inspect
from sqlalchemy.sql import func
from sqlalchemy import Float
import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc

# Imports for Lidar raster
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt

# some mapping widgets
import ipyleaflet
from ipyleaflet import Map, GeoData, Rectangle, basemaps, LayersControl, basemap_to_tiles, TileLayer, SplitMapControl, Polygon, MagnifyingGlass
import ipywidgets

# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

### Specify Inputs

In [ ]:
# Desired Magnaprobe sample dates 
depth1_date1 = datetime.date(2020,2,1) # Corresponds to ASO flight on 2/2/2020 (lidar1)
depth2_date1 = datetime.date(2020,2,12) # Corresponds to ASO flight on 2/13/2020 (lidar1)

# Desired snow pit sample dates 
density1_date1 = datetime.date(2020,1,28)
density1_date2 = datetime.date(2020,2,1) 

density2_date1 = datetime.date(2020,2,4)
density2_date2 = datetime.date(2020,2,9) 


### Define Boundingbox

In [ ]:
# How to pull out ALL point data from the database that falls within our box
bbox_WSEN = 742000, 4322000, 747000, 4325000 # EPSG 26912?
x1, y1, x2, y2 = bbox_WSEN
polygon = shapely.geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1]]) # used box() before
wkb_element = from_shape(polygon, srid=26912) # which srid is right?

### Magnaprobe Data

In [ ]:
# Instrument name 
site_name = "Grand Mesa"
instrument = "magnaprobe" 

# Get a session
engine, session = get_db(db_name)

# Find the magnaprobe PointData in the defined area within the given dates 
query_mp = session.query(PointData.geom, PointData.easting, PointData.northing, PointData.date, PointData.value, 
                    PointData.type, PointData.instrument)

# Filter by site
query_mp = query_mp.filter(PointData.site_name == site_name)
# Filter by bounding box 
query_mp = query_mp.filter(gfunc.ST_Within(PointData.geom, wkb_element))
# Filter by an instrument 
query_mp = query_mp.filter(PointData.instrument == instrument) # .in_(['magnaprobe', 'pit_ruler']))

# Slicing the dataset for specified dates 
query_mp = query_mp.filter(PointData.date == depth1_date1)

# Execute the query and convert to geopandas in one handy function
df_mp_lidar1 = query_to_geopandas(query_mp, engine) #directly pass to geopandas dataframe

# How many did were retrieved 
print(f'{len(df_mp_lidar1.index)} Magnaprobe records returned for {depth1_date1}')

session.close()

In [ ]:
# Get a session
engine, session = get_db(db_name)

# Find the magnaprobe PointData in the defined area within the given dates 
query_mp = session.query(PointData.geom, PointData.easting, PointData.northing, PointData.date, PointData.value, 
                    PointData.type, PointData.instrument)

# Filter by site
query_mp = query_mp.filter(PointData.site_name == site_name)
# Filter by bounding box 
query_mp = query_mp.filter(gfunc.ST_Within(PointData.geom, wkb_element))
# Filter by an instrument 
query_mp = query_mp.filter(PointData.instrument == instrument) # .in_(['magnaprobe', 'pit_ruler']))

# Slicing the dataset for specified dates 
df_mp_lidar2 = query_to_geopandas(query_mp, engine) #directly pass to geopandas dataframe

# Execute the query and convert to geopandas in one handy function
query_mp = query_mp.filter(PointData.date == depth2_date1)

# How many did were retrieved 
print(f'{len(df_mp_lidar2.index)} records returned for {depth2_date1}')

session.close()

In [ ]:
# print(df_mp_lidar1.sort_values(['date']).tail())

In [ ]:
# Plot sites
plt.scatter(df_mp['easting'], df_mp['northing'], alpha = 0.3, s = 0.01, c = 'red')

# View dataframe 
# df_mp = df_mp.sort_values(by = ['date'])
# print(df_mp.head())

### Snow Pit measurements

In [ ]:
# Start the session
engine, session = get_db(db_name)

# Find the snowpit LayerData in the defined area within given dates
query_sp = session.query(LayerData.geom, LayerData.easting, LayerData.northing, LayerData.date, LayerData.value, 
                         LayerData.pit_id, LayerData.type, LayerData.instrument)

# Filter by site (probably not needed)
# query_sp = query_sp.filter(LayerData.site_name == 'Grand Mesa')
# Filter by bounding box
query_sp = query_sp.filter(gfunc.ST_Within(LayerData.geom, wkb_element))
# Filter by type (pit density)
query_sp = query_sp.filter(LayerData.type == "density")
# Filter by instrument 
query_sp = query_sp.filter(LayerData.instrument==None)

# Slicing the dataset for specified dates 
query_sp = query_sp.filter(LayerData.date >= date1)
query_sp = query_sp.filter(LayerData.date <= date2)

count_meas = query_sp.count() 
print(count_meas, " Snowpit density measurements returned!\n") 

# convert to pandas dataframe  
df_sp = query_to_pandas(query_sp, engine)  

# create list of the unique dates (LayerData will have a lot of repeated dates, we only need a list per visit, not per measurement)
locations = sorted(df_sp['pit_id'].unique())
count_pits = len(locations) 
print(count_pits, " Snowpit density locations returned!\n") 

session.close()

In [ ]:
# Plot sites
plt.scatter(df_sp['easting'], df_sp['northing'])

In [ ]:
# result = session.query(LayerData.instrument).distinct().all()
# print('\nInstrument Types: ', result)

# print(avgrho_sp.dtypes)

In [ ]:
df_sp['value'] = df_sp['value'].astype(float)
avgrho_sp = df_sp[['pit_id', 'value', 'easting', 'northing']] 
avgrho_sp = avgrho_sp.groupby(['pit_id']).mean() 
print(avgrho_sp)

In [ ]:
# Plot sites
plt.scatter(df_sp['easting'], df_sp['northing'], alpha = 0.5, c = 'blue', zorder = 10, label = 'Snow Pit')
plt.scatter(df_mp['easting'], df_mp['northing'], alpha = 0.5, c = 'red', zorder = 1, label = 'Magnaprobe')
plt.legend()

In [ ]:
# avgrho_sp = df_sp[['easting', 'northing', 'pit_id', 'value']]
# print(avgrho_sp.head())

# avgrho_sp = df_sp.groupby(['pit_id']).mean()
# print(avgrho_sp.head())


# Average Snowpit density measurements
# Request the average (avg) of Layer data casted as a float. We have to cast to a float in the layer table because all main values are stored as a string to
# ...accommodate the hand hardness.
qry = session.query(func.avg(LayerData.value.cast(Float)))
# # Filter our query only to density
query_sp = query_sp.filter(gfunc.ST_Within(LayerData.geom, wkb_element))
qry = qry.filter(LayerData.type=='density')
qry = qry.filter(LayerData.instrument==None)
query_sp = query_sp.filter(LayerData.date >= date1)
query_sp = query_sp.filter(LayerData.date <= date2)

In [ ]:
# Request the data
rho_avg_all = qry.all()
# Request the Average Density of Just 1S1
rho_avg_1s1 = qry.filter(LayerData.site_id == '2N12').limit(1)
# This is a gotcha. The data in layer data only is stored as a string to accommodate the hand hardness values
print(f"Average density of all pits is {rho_avg_all[0][0]:0.0f} kg/m3")
print(f"Average density of pit 2N12 is {rho_avg_1s1[0][0]:0.0f} kg/m3")

# Cast Densities to float
rho_avg_all = float(rho_avg_all[0][0])
rho_avg_1s1 = float(rho_avg_1s1[0][0])

# # Can use LayerData.pit_id to average over

### GPR Data

In [ ]:
# # Get the GPR Dates

# # Start the session
# engine, session = get_db(db_name)

# # Define the GPR instrument identifier
# bsu_gpr_instrument_name = "pulse EKKO Pro multi-polarization 1 GHz GPR"

# # Query the Pointdata to isolate by GPR instrument
# query = session.query(PointData.date)
# query = query.filter(PointData.instrument == bsu_gpr_instrument_name)

# # convert to pandas dataframe  
# df = query_to_pandas(query, engine)  

# # create list of the unique dates (LayerData will have a lot of repeated dates, we only need a list per visit, not per measurement)
# GPR_dates = sorted(df['date'].unique())

# # Print the given dates
# print(GPR_dates)

# session.close()

In [ ]:
# # Start the session
# engine, session = get_db(db_name)

# # Limit the size of the GPR data for initial processing and plot
# query = session.query(PointData)
# query = query.filter(PointData.instrument == bsu_gpr_instrument_name)
# query = query.distinct().order_by(func.random()).limit(100) #all()

# bsu_gpr_sample = query_to_geopandas(query, engine)

# bsu_gpr_sample.plot()

# session.close()

### Lidar Raster (In-Progress)

In [ ]:
# Start the session
engine, session = get_db(db_name)

query = session.query(ImageData.units)
query = query.filter(ImageData.observers == "ASO Inc.")
query = query.filter(ImageData.type == "depth")
query = query.distinct()
result = query.all()
print(result)

session.close()